<a href="https://colab.research.google.com/github/parjhanya/Image_Classification/blob/master/Image_Classification_Custom_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def append_ext(fn):
    return fn+".png"
traindf=pd.read_csv('/content/drive/My Drive/Image Classification/Extracted/train/trainLabels.csv',dtype=str)
testdf=pd.read_csv('/content/drive/My Drive/Image Classification/Extracted/object-recognition-predictions.csv',dtype=str)
traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory='/content/drive/My Drive/Image Classification/Extracted/train/train',
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 3902 validated image filenames belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 44798 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory='/content/drive/My Drive/Image Classification/Extracted/train/train',
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 1300 validated image filenames belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 44798 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)

In [ ]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory='/content/drive/My Drive/Image Classification/Extracted/Object_classification_test_data',
x_col="id",
y_col="prediction",
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 0 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1365 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
121/121 [==============================] - 3067s 25s/step - loss: 2.2792 - accuracy: 0.1292 - val_loss: 2.2596 - val_accuracy: 0.1461
Epoch 2/10
121/121 [==============================] - 16s 130ms/step - loss: 2.1156 - accuracy: 0.2121 - val_loss: 2.1284 - val_accuracy: 0.2737
Epoch 3/10
121/121 [==============================] - 13s 104ms/step - loss: 1.9810 - accuracy: 0.2620 - val_loss: 2.0196 - val_accuracy: 0.3249
Epoch 4/10
121/121 [==============================] - 13s 105ms/step - loss: 1.8957 - accuracy: 0.3070 - val_loss: 1.7389 - val_accuracy: 0.3644
Epoch 5/10
121/121 [==============================] - 12s 103ms/step - loss: 1.8167 - accuracy: 0.3377 - val_loss: 1.8292 - val_accuracy: 0.3573
Epoch 6/10
121/121 [==============================] - 13s 104ms/step - loss: 1.7588 - accuracy: 0.3545 - val_loss: 1.6685 - val_accuracy: 0.3872
Epoch 7/10
121/121 [==============================] - 12s 102ms/step - loss: 1.7234 - accuracy: 0.3615 - val_loss: 1.5454 - val_ac

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)